In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle
import os
import scikeras 
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
import keras_tuner as kt
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
import pickle
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data,housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full,y_train_full, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


2024-05-28 21:25:14.983994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from scipy.stats import reciprocal
learning_rate_values = reciprocal(3e-4, 3e-2).rvs(1000).tolist()
len(learning_rate_values)

1000

In [3]:
from scipy.stats import randint
param_distribs  = {
    "model__n_hidden": list(range(0, 4)),
    "model__n_neurons": list(range(1, 101)),
    "model__learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
    "model__optimizer": ['adam', 'sgd', 'nesterov']
}


In [4]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train[0].shape))

    for n_l in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation = 'relu'))

    model.add(tf.keras.layers.Dense(1))
    
    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    else:
        raise ValueError("Unsupported optimizer type. Choose from 'sgd', 'nesterov', 'momentum', or 'adam'.")

    

    model.compile(loss= 'mse', optimizer=optimizer,metrics=["mse"])
    return model


# ...
# model.compile(...)
#return model


In [5]:
import scikeras
from scikeras.wrappers import KerasRegressor

In [6]:
import tensorflow as tf
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])


ModuleNotFoundError: No module named 'keras.api._v2'

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rnd_search_cv = RandomizedSearchCV(keras_reg,param_distribs,
                                   n_iter=5,cv=3,verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid,y_valid), verbose=0)



In [ ]:
best_params = rnd_search_cv.best_params_

In [ ]:
import pickle
with open('rnd_search_params.pkl', 'wb') as f:
    pickle.dump(best_params, f)

with open('rnd_search_scikeras.pkl', 'wb') as f:
    pickle(rnd_search, f)

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model_kt(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3, default=2)
    n_neurons = hp.Int('n_neurons', min_value=1, max_value=100, default=30)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 3e-3, 1e-2, 3e-2])
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'nesterov'])

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train[0].shape))

    for n_l in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation = 'relu'))

    model.add(tf.keras.layers.Dense(1))
    
    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    else:
        raise ValueError("Unsupported optimizer type. Choose from 'sgd', 'nesterov', 'momentum', or 'adam'.")
    model.compile(loss= 'mse', optimizer = optimizer,metrics=["mse"])
    return model
# model.compile(...)
  #  return model


In [ ]:
random_search_tuner = kt.RandomSearch(
build_model_kt, objective="val_mse", max_trials=10, overwrite=True,
directory="my_california_housing", project_name="my_rnd_search", seed=42)

In [ ]:
root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = tf.keras.callbacks.TensorBoard(root_logdir)


In [ ]:
random_search_tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[tb, es])

random_search_tuner.results_summary()

In [11]:
######
with open('kt_search_params.pkl', 'wb') as f:
    pickle.dump(random_search_tuner.get_best_hyperparameters()[0].values, f)

random_search_tuner.get_best_models(num_models=1)[0].save('kt_best_model.keras')

NameError: name 'random_search_tuner' is not defined

In [10]:
print(tf.__version__)

2.12.0
